In [ ]:
import requests
import webbrowser
import pandas as pd
import upstox_client
from upstox_client.rest import ApiException
from datetime import datetime, timedelta

def get_authorization_code(client_id, redirect_uri):
    auth_url = f'https://api.upstox.com/v2/login/authorization/dialog?response_type=code&client_id={client_id}&redirect_uri={redirect_uri}'
    print("Please visit the following URL to authorize the application:")
    print(auth_url)
    webbrowser.open(auth_url)
    authorization_code = input("Enter the authorization code received after authorization: ")
    return authorization_code

def get_access_token(client_id, client_secret, redirect_uri, authorization_code):
    token_url = 'https://api.upstox.com/v2/login/authorization/token'
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    data = {
        'code': authorization_code,
        'client_id': client_id,
        'client_secret': client_secret,
        'redirect_uri': redirect_uri,
        'grant_type': 'authorization_code',
    }
    response = requests.post(token_url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        print("Error fetching access token:", response.json())
        return None

def fetch_user_profile(access_token):
    profile_url = 'https://api.upstox.com/v2/user/profile'
    headers = {
        'Accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(profile_url, headers=headers)
    return response

def fetch_fund_and_margin(access_token):
    url = 'https://api.upstox.com/v2/user/get-funds-and-margin'
    headers = {
        'Accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, headers=headers)
    return response

def get_instrument_key(segment, trading_symbol):
    filtered = scripts[(scripts["segment"].isin(segment)) & (scripts["trading_symbol"] == trading_symbol)]
    if not filtered.empty:
        return filtered["instrument_key"].values[0]
    else:
        return "Instrument not found"

def prompt_user_for_segment_and_trading_symbols():
    segments = input("Enter segments separated by commas: ").split(',')
    segments = [segment.strip() for segment in segments]
    
    trading_symbols = input("Enter trading symbols separated by commas (SYMBOL STRIKEPRICE PE/CE DD MMM YY): ").split(',')
    trading_symbols = [symbol.strip() for symbol in trading_symbols]
    
    return segments, trading_symbols

def fetch_historical_candle_data(instrument_key, access_token):
    # Initialize the API client
    configuration = upstox_client.Configuration()
    configuration.access_token = access_token
    api_instance = upstox_client.HistoryApi(upstox_client.ApiClient(configuration))
    
    # Determine dates
    to_date = datetime.now().strftime('%Y-%m-%d')
    from_date = (datetime.now() - timedelta(days=3)).strftime('%Y-%m-%d')

    try:
        api_response = api_instance.get_historical_candle_data1(
            instrument_key,
            interval='1minute',
            to_date=to_date,
            from_date=from_date,
            api_version='2.0'
        )
        print("Historical Candle Data:")
        print(api_response)
    except ApiException as e:
        print("Exception when calling HistoryApi->get_historical_candle_data: %s\n" % e)

def fetch_intraday_data(instrument_key, access_token):
    url = f'https://api.upstox.com/v2/historical-candle/intraday/{instrument_key}/1minute'
    headers = {
        'Accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            print("Raw API Response:", data)
            if data.get('status') == 'success':
                candles = data.get('data', {}).get('candles', [])
                if candles:
                    print("Data fetched successfully.")
                    # Print data for debugging purposes
                    for candle in candles:
                        print(candle)
                else:
                    print("NO DATA AVAILABLE")
            else:
                print("Error in API response:", data)
        else:
            print(f"Error: {response.status_code} - {response.text}")
    except Exception as e:
        print("Exception occurred:", e)

def fetch_market_feed(instrument_keys, access_token):
    url = f'https://api.upstox.com/v2/market-quote/ltp?instrument_key={",".join(instrument_keys)}'
    headers = {
        'Accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            print("Raw API Response:", data)
            if data.get('status') == 'success':
                market_feed_data = data.get('data', {})
                if market_feed_data:
                    for key, value in market_feed_data.items():
                        print(f"Instrument Key: {key} - Last Price: {value.get('last_price', 'N/A')}")
                else:
                    print("NO FEED AVAILABLE")
            else:
                print("Error in API response:", data)
        else:
            print(f"Error: {response.status_code} - {response.text}")
    except Exception as e:
        print("Exception occurred:", e)

import pandas as pd

def market_data_streamer(instrument_keys, access_token):
    def on_message(message):
        # Add the received message to the buffer
        buffer.append(message)

        # Process each feed in the message
        feeds = message.get('feeds', {})
        for key, value in feeds.items():
            print(f"\nFeed: {key}")
            market_ohlc = value.get('ff', {}).get('marketOHLC', {})
            if market_ohlc:
                ohlc_data = market_ohlc.get('ohlc', [])
                if ohlc_data:
                    # Convert to DataFrame for better tabulated display
                    df = pd.DataFrame(ohlc_data)
                    print("Market OHLC Data:")
                    print(df.to_string(index=False))
                else:
                    print("No OHLC data available.")
            else:
                print("No marketOHLC data available.")

    # Configure the API client with the access token
    configuration = upstox_client.Configuration()
    configuration.access_token = access_token

    try:
        # Initialize the MarketDataStreamer
        streamer = upstox_client.MarketDataStreamer(
            upstox_client.ApiClient(configuration), 
            instrument_keys,  # List of instrument keys to stream
            "full"  # Specify the type of data you want to stream (e.g., "full" for all available data)
        )

        # Register the callback for handling incoming messages
        buffer = []  # Initialize a buffer to store incoming messages
        streamer.on("message", on_message)

        # Connect to the market data stream
        streamer.connect()

        print("Streaming started. Press Ctrl+C to stop.")

        import time
        last_time = time.time()
        while True:
            current_time = time.time()
            if current_time - last_time >= 120:  # Check if 120 seconds have passed
                last_time = current_time
                # Print buffered messages
                if buffer:
                    print("Buffered Messages at:", datetime.now())
                    for message in buffer:
                        print("Raw Message:", message)
                        # The on_message function already handles data extraction and printing
                    buffer.clear()  # Clear the buffer after printing
                else:
                    print("No new messages to display.")
            time.sleep(60)  # Sleep for 60 seconds to avoid excessive CPU usage

    except Exception as e:
        print("Exception occurred in streaming:", e)

def main():
    global scripts
    scripts = pd.read_json("https://assets.upstox.com/market-quote/instruments/exchange/complete.json.gz")

    client_id = "1c6b1068-cfee-4cee-b2a7-4580ee379dd0"
    client_secret = "yhcwzo46ek"
    redirect_uri = "https://api.upstox.com/v2/login"

    authorization_code = get_authorization_code(client_id, redirect_uri)
    access_token = get_access_token(client_id, client_secret, redirect_uri, authorization_code)

    if access_token:
        profile_response = fetch_user_profile(access_token)
        print(f"User Profile Status Code: {profile_response.status_code}")
        print("User Profile:", profile_response.json())
        
        fund_margin_response = fetch_fund_and_margin(access_token)
        print(f"Fund and Margin Status Code: {fund_margin_response.status_code}")
        print("Fund and Margin Details:", fund_margin_response.json())
        
        segments, trading_symbols = prompt_user_for_segment_and_trading_symbols()
        instrument_keys = []
        for symbol in trading_symbols:
            instrument_key = get_instrument_key(segments, symbol)
            print(f"Instrument Key for '{symbol}' in segments {segments}: {instrument_key}")
            if instrument_key != "Instrument not found":
                instrument_keys.append(instrument_key)

        if instrument_keys:
            fetch_historical_candle_data(instrument_keys[0], access_token)
            fetch_intraday_data(instrument_keys[0], access_token)
            fetch_market_feed(instrument_keys, access_token)
            market_data_streamer(instrument_keys, access_token)
        else:
            print("No valid instrument keys found. Cannot proceed with data fetching and streaming.")

if __name__ == "__main__":
    main()


Please visit the following URL to authorize the application:
https://api.upstox.com/v2/login/authorization/dialog?response_type=code&client_id=1c6b1068-cfee-4cee-b2a7-4580ee379dd0&redirect_uri=https://api.upstox.com/v2/login


Enter the authorization code received after authorization:  vRJvq9


User Profile Status Code: 200
User Profile: {'status': 'success', 'data': {'email': 'pritish.dash@gmail.com', 'exchanges': ['NSE', 'BSE', 'BFO', 'CDS', 'NFO', 'BCD'], 'products': ['OCO', 'D', 'CO', 'I'], 'broker': 'UPSTOX', 'user_id': '866215', 'user_name': 'PRITISH DASH', 'order_types': ['MARKET', 'LIMIT', 'SL', 'SL-M'], 'user_type': 'individual', 'poa': False, 'is_active': True}}
Fund and Margin Status Code: 200
Fund and Margin Details: {'status': 'success', 'data': {'commodity': {'used_margin': 0.0, 'payin_amount': 0.0, 'span_margin': 0.0, 'adhoc_margin': 0.0, 'notional_cash': 0.0, 'available_margin': 0.0, 'exposure_margin': 0.0}, 'equity': {'used_margin': 0.0, 'payin_amount': 0.0, 'span_margin': 0.0, 'adhoc_margin': 0.0, 'notional_cash': 0.0, 'available_margin': 4000.0, 'exposure_margin': 0.0}}}


Enter segments separated by commas:  NSE_FO
Enter trading symbols separated by commas (SYMBOL STRIKEPRICE PE/CE DD MMM YY):  NIFTY 24400 CE 08 AUG 24


Instrument Key for 'NIFTY 24400 CE 08 AUG 24' in segments ['NSE_FO']: NSE_FO|51776
Historical Candle Data:
{'data': {'candles': [['2024-08-06T15:29:00+05:30',
                       26.85,
                       27.25,
                       25.55,
                       27,
                       291775,
                       3802325],
                      ['2024-08-06T15:28:00+05:30',
                       26.9,
                       27.45,
                       26.6,
                       26.85,
                       266350,
                       3802325],
                      ['2024-08-06T15:27:00+05:30',
                       25.95,
                       27.2,
                       25.7,
                       26.9,
                       257550,
                       3831650],
                      ['2024-08-06T15:26:00+05:30',
                       24.65,
                       26,
                       24.6,
                       25.9,
                       183